In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout, Add, LSTM, GRU, RNN, LayerNormalization, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Layer
from keras import backend as K
from keras.optimizers import Adam,SGD
import tensorflow as tf
from keras import Model, regularizers, activations
from keras.constraints import Constraint
import pickle
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist

In [2]:
class MyLRNNCell(tf.keras.layers.Layer):

    def __init__(self, units, eps=0.01, gamma=0.01, beta=0.8, alpha=1, **kwargs):
        self.units = units
        self.state_size = units
        self.I = tf.eye(units)
        self.eps = eps
        self.gamma = gamma
        self.beta = beta
        self.alpha = alpha
        super(MyLRNNCell, self).__init__(**kwargs)

    def build(self, input_shape):
        self.C = self.add_weight(shape=(self.units, self.units),
                                      initializer='random_normal',
                                      name='C',
                                      trainable=True)
        self.B = self.add_weight(shape=(self.units, self.units),
                                                initializer='random_normal',
                                                name='B',
                                                trainable=True)
        self.U = self.add_weight(shape=(input_shape[-1], self.units),
                                                initializer='random_normal',
                                                name='U',
                                                trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                    initializer='zeros',
                                    name='b',
                                    trainable=True)
        self.built = True

    def call(self, inputs, states):
        prev_h = states[0]

        A = self.beta * (self.B - tf.transpose(self.B)) + (1 - self.beta) * (self.B + tf.transpose(self.B)) - self.gamma * self.I
        W = self.beta * (self.C - tf.transpose(self.C)) + (1 - self.beta) * (self.C + tf.transpose(self.C)) - self.gamma * self.I

        h = prev_h + self.eps * self.alpha * K.dot(prev_h, A) + self.eps * tf.nn.tanh(K.dot(prev_h, W) + K.dot(inputs, self.U) + self.b)
        return h, [h]

    def get_config(self):
        config = super(MyLRNNCell, self).get_config()
        config.update({"units": self.units, "eps":self.eps, "gamma":self.gamma, "beta":self.beta, "alpha":self.alpha})
        return config

In [3]:
def add_salt_and_pepper_noise(images, amount):
    noisy_images = images.copy()
    num_images = images.shape[0]
    num_pixels = images.shape[1] * images.shape[2]
    num_salt = np.ceil(amount * num_pixels)
    for i in range(num_images):
        # Add salt noise
        salt_indices = np.random.choice(num_pixels, size=int(num_salt), replace=False)
        noisy_images[i].flat[salt_indices] = 1
        
        # Add pepper noise
        pepper_indices = np.random.choice(num_pixels, size=int(num_salt), replace=False)
        noisy_images[i].flat[pepper_indices] = 0
    return noisy_images

In [4]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, SimpleRNN
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist, cifar10

tf.random.set_seed(42)

training_acc = [0]*10
test_acc = [0]*10
noise = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for i in range(10):
    # load mnist dataset
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    # compute the number of labels
    num_labels = len(np.unique(y_train))

    # convert to one-hot vector
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    # resize and normalize
    image_size = X_train.shape[1]
    X_train = np.reshape(X_train,[-1, image_size, image_size])
    X_test = np.reshape(X_test,[-1, image_size, image_size])
    X_train = X_train.astype('float32') / 255
    X_test = X_test.astype('float32') / 255
    
    X_train = add_salt_and_pepper_noise(X_train, noise[i])

    input = Input(shape=(X_train.shape[1],X_train.shape[2]))
    x = RNN(MyLRNNCell(units=64),return_sequences=True)(input)
    x = RNN(MyLRNNCell(units=64),return_sequences=False)(x)
    x = Dense(num_labels, activation='softmax')(x)
    model = Model(input, x)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=100, batch_size=128, validation_split=0.25, verbose=2)

    training_acc[i] = history.history['val_accuracy'][-1]
    print(training_acc)
    
    loss, acc = model.evaluate(X_test, y_test, batch_size=128)

    test_acc[i] = acc
    print(test_acc)

    name = 'lrnn_64_'
    name = name + str(noise[i]) + '.h5'
    model.save(name)

model.summary()

Epoch 1/100
352/352 - 17s - loss: 2.0235 - accuracy: 0.2712 - val_loss: 1.7246 - val_accuracy: 0.4145 - 17s/epoch - 48ms/step
Epoch 2/100
352/352 - 8s - loss: 1.5325 - accuracy: 0.4783 - val_loss: 1.3852 - val_accuracy: 0.5292 - 8s/epoch - 24ms/step
Epoch 3/100
352/352 - 9s - loss: 1.2851 - accuracy: 0.5650 - val_loss: 1.1702 - val_accuracy: 0.6079 - 9s/epoch - 25ms/step
Epoch 4/100
352/352 - 9s - loss: 1.0944 - accuracy: 0.6304 - val_loss: 0.9981 - val_accuracy: 0.6643 - 9s/epoch - 25ms/step
Epoch 5/100
352/352 - 8s - loss: 0.9519 - accuracy: 0.6751 - val_loss: 0.8838 - val_accuracy: 0.7011 - 8s/epoch - 24ms/step
Epoch 6/100
352/352 - 9s - loss: 0.8410 - accuracy: 0.7184 - val_loss: 0.7824 - val_accuracy: 0.7439 - 9s/epoch - 25ms/step
Epoch 7/100
352/352 - 9s - loss: 0.7518 - accuracy: 0.7524 - val_loss: 0.7032 - val_accuracy: 0.7741 - 9s/epoch - 25ms/step
Epoch 8/100
352/352 - 9s - loss: 0.6755 - accuracy: 0.7810 - val_loss: 0.6340 - val_accuracy: 0.7979 - 9s/epoch - 25ms/step
Epoch 

Epoch 67/100
352/352 - 8s - loss: 0.1166 - accuracy: 0.9648 - val_loss: 0.1394 - val_accuracy: 0.9583 - 8s/epoch - 24ms/step
Epoch 68/100
352/352 - 8s - loss: 0.1159 - accuracy: 0.9647 - val_loss: 0.1408 - val_accuracy: 0.9563 - 8s/epoch - 24ms/step
Epoch 69/100
352/352 - 8s - loss: 0.1139 - accuracy: 0.9651 - val_loss: 0.1384 - val_accuracy: 0.9593 - 8s/epoch - 24ms/step
Epoch 70/100
352/352 - 8s - loss: 0.1135 - accuracy: 0.9652 - val_loss: 0.1366 - val_accuracy: 0.9594 - 8s/epoch - 24ms/step
Epoch 71/100
352/352 - 8s - loss: 0.1119 - accuracy: 0.9667 - val_loss: 0.1341 - val_accuracy: 0.9588 - 8s/epoch - 23ms/step
Epoch 72/100
352/352 - 8s - loss: 0.1101 - accuracy: 0.9664 - val_loss: 0.1409 - val_accuracy: 0.9582 - 8s/epoch - 23ms/step
Epoch 73/100
352/352 - 8s - loss: 0.1096 - accuracy: 0.9671 - val_loss: 0.1358 - val_accuracy: 0.9596 - 8s/epoch - 22ms/step
Epoch 74/100
352/352 - 8s - loss: 0.1070 - accuracy: 0.9674 - val_loss: 0.1310 - val_accuracy: 0.9611 - 8s/epoch - 24ms/step


C:\Users\zihaow19\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/100
352/352 - 17s - loss: 2.0735 - accuracy: 0.2520 - val_loss: 1.8383 - val_accuracy: 0.3419 - 17s/epoch - 49ms/step
Epoch 2/100
352/352 - 8s - loss: 1.7180 - accuracy: 0.3933 - val_loss: 1.5943 - val_accuracy: 0.4450 - 8s/epoch - 24ms/step
Epoch 3/100
352/352 - 8s - loss: 1.4977 - accuracy: 0.4811 - val_loss: 1.4147 - val_accuracy: 0.5107 - 8s/epoch - 24ms/step
Epoch 4/100
352/352 - 8s - loss: 1.3666 - accuracy: 0.5246 - val_loss: 1.3104 - val_accuracy: 0.5407 - 8s/epoch - 24ms/step
Epoch 5/100
352/352 - 8s - loss: 1.2659 - accuracy: 0.5629 - val_loss: 1.2156 - val_accuracy: 0.5792 - 8s/epoch - 24ms/step
Epoch 6/100
352/352 - 8s - loss: 1.1650 - accuracy: 0.6061 - val_loss: 1.1121 - val_accuracy: 0.6250 - 8s/epoch - 24ms/step
Epoch 7/100
352/352 - 8s - loss: 1.0745 - accuracy: 0.6362 - val_loss: 1.0315 - val_accuracy: 0.6514 - 8s/epoch - 24ms/step
Epoch 8/100
352/352 - 8s - loss: 1.0019 - accuracy: 0.6619 - val_loss: 0.9674 - val_accuracy: 0.6731 - 8s/epoch - 24ms/step
Epoch 

Epoch 67/100
352/352 - 8s - loss: 0.2607 - accuracy: 0.9166 - val_loss: 0.2874 - val_accuracy: 0.9080 - 8s/epoch - 22ms/step
Epoch 68/100
352/352 - 8s - loss: 0.2591 - accuracy: 0.9172 - val_loss: 0.2782 - val_accuracy: 0.9123 - 8s/epoch - 22ms/step
Epoch 69/100
352/352 - 8s - loss: 0.2559 - accuracy: 0.9179 - val_loss: 0.2850 - val_accuracy: 0.9090 - 8s/epoch - 22ms/step
Epoch 70/100
352/352 - 8s - loss: 0.2540 - accuracy: 0.9192 - val_loss: 0.2778 - val_accuracy: 0.9113 - 8s/epoch - 23ms/step
Epoch 71/100
352/352 - 8s - loss: 0.2529 - accuracy: 0.9187 - val_loss: 0.2730 - val_accuracy: 0.9143 - 8s/epoch - 24ms/step
Epoch 72/100
352/352 - 8s - loss: 0.2491 - accuracy: 0.9202 - val_loss: 0.2744 - val_accuracy: 0.9131 - 8s/epoch - 23ms/step
Epoch 73/100
352/352 - 8s - loss: 0.2482 - accuracy: 0.9200 - val_loss: 0.2709 - val_accuracy: 0.9145 - 8s/epoch - 24ms/step
Epoch 74/100
352/352 - 8s - loss: 0.2454 - accuracy: 0.9220 - val_loss: 0.2693 - val_accuracy: 0.9127 - 8s/epoch - 24ms/step


...

Epoch 45/100
352/352 - 9s - loss: 2.1673 - accuracy: 0.2102 - val_loss: 2.1850 - val_accuracy: 0.2010 - 9s/epoch - 25ms/step
Epoch 46/100
352/352 - 8s - loss: 2.1655 - accuracy: 0.2109 - val_loss: 2.1855 - val_accuracy: 0.2001 - 8s/epoch - 24ms/step
Epoch 47/100
352/352 - 9s - loss: 2.1649 - accuracy: 0.2119 - val_loss: 2.1724 - val_accuracy: 0.2044 - 9s/epoch - 24ms/step
Epoch 48/100
352/352 - 9s - loss: 2.1642 - accuracy: 0.2138 - val_loss: 2.1703 - val_accuracy: 0.2061 - 9s/epoch - 24ms/step
Epoch 49/100
352/352 - 8s - loss: 2.1629 - accuracy: 0.2134 - val_loss: 2.1716 - val_accuracy: 0.2075 - 8s/epoch - 24ms/step
Epoch 50/100
352/352 - 9s - loss: 2.1629 - accuracy: 0.2138 - val_loss: 2.1737 - val_accuracy: 0.2032 - 9s/epoch - 24ms/step
Epoch 51/100
352/352 - 8s - loss: 2.1619 - accuracy: 0.2137 - val_loss: 2.1715 - val_accuracy: 0.2057 - 8s/epoch - 24ms/step
Epoch 52/100
352/352 - 8s - loss: 2.1621 - accuracy: 0.2137 - val_loss: 2.1683 - val_accuracy: 0.2081 - 8s/epoch - 22ms/step


352/352 - 15s - loss: 2.2880 - accuracy: 0.1354 - val_loss: 2.2932 - val_accuracy: 0.1257 - 15s/epoch - 42ms/step
Epoch 8/100
352/352 - 15s - loss: 2.2863 - accuracy: 0.1371 - val_loss: 2.2884 - val_accuracy: 0.1323 - 15s/epoch - 43ms/step
Epoch 9/100
352/352 - 15s - loss: 2.2852 - accuracy: 0.1380 - val_loss: 2.2890 - val_accuracy: 0.1325 - 15s/epoch - 43ms/step
Epoch 10/100
352/352 - 15s - loss: 2.2841 - accuracy: 0.1391 - val_loss: 2.2858 - val_accuracy: 0.1353 - 15s/epoch - 44ms/step
Epoch 11/100
352/352 - 15s - loss: 2.2825 - accuracy: 0.1395 - val_loss: 2.2851 - val_accuracy: 0.1357 - 15s/epoch - 42ms/step
Epoch 12/100
352/352 - 14s - loss: 2.2812 - accuracy: 0.1414 - val_loss: 2.2867 - val_accuracy: 0.1363 - 14s/epoch - 40ms/step
Epoch 13/100
352/352 - 15s - loss: 2.2802 - accuracy: 0.1430 - val_loss: 2.2831 - val_accuracy: 0.1379 - 15s/epoch - 43ms/step
Epoch 14/100
352/352 - 15s - loss: 2.2789 - accuracy: 0.1435 - val_loss: 2.2829 - val_accuracy: 0.1355 - 15s/epoch - 42ms/step

Epoch 72/100
352/352 - 15s - loss: 2.2602 - accuracy: 0.1592 - val_loss: 2.2728 - val_accuracy: 0.1461 - 15s/epoch - 42ms/step
Epoch 73/100
352/352 - 15s - loss: 2.2598 - accuracy: 0.1582 - val_loss: 2.2724 - val_accuracy: 0.1470 - 15s/epoch - 42ms/step
Epoch 74/100
352/352 - 15s - loss: 2.2600 - accuracy: 0.1573 - val_loss: 2.2727 - val_accuracy: 0.1464 - 15s/epoch - 42ms/step
Epoch 75/100
352/352 - 15s - loss: 2.2590 - accuracy: 0.1591 - val_loss: 2.2723 - val_accuracy: 0.1475 - 15s/epoch - 42ms/step
Epoch 76/100
352/352 - 15s - loss: 2.2588 - accuracy: 0.1579 - val_loss: 2.2724 - val_accuracy: 0.1459 - 15s/epoch - 41ms/step
Epoch 77/100
352/352 - 15s - loss: 2.2588 - accuracy: 0.1578 - val_loss: 2.2721 - val_accuracy: 0.1461 - 15s/epoch - 42ms/step
Epoch 78/100
352/352 - 14s - loss: 2.2584 - accuracy: 0.1597 - val_loss: 2.2724 - val_accuracy: 0.1491 - 14s/epoch - 40ms/step
Epoch 79/100
352/352 - 14s - loss: 2.2581 - accuracy: 0.1586 - val_loss: 2.2753 - val_accuracy: 0.1457 - 14s/ep

Epoch 33/100
352/352 - 14s - loss: 2.2962 - accuracy: 0.1217 - val_loss: 2.3026 - val_accuracy: 0.1117 - 14s/epoch - 39ms/step
Epoch 34/100
352/352 - 14s - loss: 2.2963 - accuracy: 0.1209 - val_loss: 2.3031 - val_accuracy: 0.1115 - 14s/epoch - 41ms/step
Epoch 35/100
352/352 - 14s - loss: 2.2963 - accuracy: 0.1220 - val_loss: 2.3028 - val_accuracy: 0.1101 - 14s/epoch - 40ms/step
Epoch 36/100
352/352 - 14s - loss: 2.2961 - accuracy: 0.1219 - val_loss: 2.3024 - val_accuracy: 0.1137 - 14s/epoch - 40ms/step
Epoch 37/100
352/352 - 14s - loss: 2.2959 - accuracy: 0.1234 - val_loss: 2.3030 - val_accuracy: 0.1146 - 14s/epoch - 40ms/step
Epoch 38/100
352/352 - 14s - loss: 2.2959 - accuracy: 0.1214 - val_loss: 2.3025 - val_accuracy: 0.1146 - 14s/epoch - 40ms/step
Epoch 39/100
352/352 - 14s - loss: 2.2958 - accuracy: 0.1223 - val_loss: 2.3023 - val_accuracy: 0.1159 - 14s/epoch - 40ms/step
Epoch 40/100
352/352 - 14s - loss: 2.2956 - accuracy: 0.1235 - val_loss: 2.3028 - val_accuracy: 0.1121 - 14s/ep

Epoch 98/100
352/352 - 14s - loss: 2.2913 - accuracy: 0.1301 - val_loss: 2.3039 - val_accuracy: 0.1151 - 14s/epoch - 41ms/step
Epoch 99/100
352/352 - 15s - loss: 2.2912 - accuracy: 0.1285 - val_loss: 2.3030 - val_accuracy: 0.1158 - 15s/epoch - 41ms/step
Epoch 100/100
352/352 - 15s - loss: 2.2910 - accuracy: 0.1289 - val_loss: 2.3030 - val_accuracy: 0.1149 - 15s/epoch - 41ms/step
[0.9639999866485596, 0.9260666370391846, 0.8644000291824341, 0.7670666575431824, 0.6362000107765198, 0.4947333335876465, 0.33533334732055664, 0.22253333032131195, 0.1509999930858612, 0.11486666649580002]
79/79 [==============================] - 1s 13ms/step - loss: 2.0072 - accuracy: 0.2958
[0.9659000039100647, 0.8774999976158142, 0.7168999910354614, 0.5902000069618225, 0.5343999862670898, 0.4555000066757202, 0.36980000138282776, 0.33480000495910645, 0.38659998774528503, 0.29580000042915344]
Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Sh

In [5]:
from tensorflow.python.framework.convert_to_constants import  convert_variables_to_constants_v2_as_graph

def get_flops(model):
    concrete = tf.function(lambda inputs: model(inputs))
    concrete_func = concrete.get_concrete_function(
        [tf.TensorSpec([1, *inputs.shape[1:]]) for inputs in model.inputs])
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(concrete_func, lower_control_flow=False)
    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name='')
        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        flops = tf.compat.v1.profiler.profile(graph=graph, run_meta=run_meta, cmd="op", options=opts)
        return flops.total_float_ops

print("The FLOPs is:{}".format(get_flops(model)) ,flush=True )

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
The FLOPs is:161340
